In [ ]:

import sys
import os
import pandas as pd
# insert at 1, 0 is the script path (or '' in REPL)
sys.path.append('scripts')
from helper import read_file

In [ ]:
import helper
print(helper.__file__)


In [ ]:
consolidated_skus_file = '00_Data_for_Fatos.csv'
file_extension = 'csv'

In [ ]:
df = pd.read_csv(consolidated_skus_file)
    
df.head()

In [ ]:
df['Week'] =pd.to_datetime(df.Week)

In [ ]:
df = df.sort_values('Week',ascending=True)

In [ ]:
timestamp_col = 'Week'

In [ ]:
sku_list = df.columns.tolist()

In [ ]:
sku_list

In [ ]:
sku_df = df[[timestamp_col]]

In [ ]:
sku_df['Sku'] = 'sku01'
sku_df['Sales'] = df['sku01']

In [ ]:
sku_list.remove(timestamp_col)

In [ ]:
sku_list

In [ ]:
data_folder = 'dataskus'

In [ ]:
df.head()

In [ ]:
for sku in sku_list:
    sku_df = df[[timestamp_col]]
    sku_df['Sku'] = sku
    sku_df['Sales'] = df[sku]
    sku_df.to_csv(data_folder + '/' + sku + '.csv', index = False)

In [ ]:
split_date = '18/05/2020'
timestamp_column = timestamp_col
target_path = data_folder

In [ ]:
from helper import split_data

In [ ]:
train_path, inference_path = split_data(target_path, timestamp_column, split_date)

In [ ]:
from azureml.core.workspace import Workspace

ws = Workspace.from_config('config.json')

# Take a look at Workspace
ws.get_details()

In [ ]:
print(target_path)

In [ ]:
# Connect to default datastore
datastore = ws.get_default_datastore()

# Upload train data
ds_train_path = target_path + '_train'
datastore.upload(src_dir=train_path, target_path=ds_train_path, overwrite=True)

# Upload inference data
ds_inference_path = target_path + '_inference'
datastore.upload(src_dir=inference_path, target_path=ds_inference_path, overwrite=True)

In [ ]:
from azureml.core.dataset import Dataset

# Create file datasets
ds_train = Dataset.File.from_files(path=datastore.path(ds_train_path), validate=False)
ds_inference = Dataset.File.from_files(path=datastore.path(ds_inference_path), validate=False)

# Register the file datasets
dataset_name = 'datasku'
train_dataset_name = dataset_name + '_train'
inference_dataset_name = dataset_name + '_inference'
ds_train.register(ws, train_dataset_name, create_new_version=True)
ds_inference.register(ws, inference_dataset_name, create_new_version=True)